<h1>Data analysis report for bike company</h1>
<i>Konsta Nyman</i>

In [1]:
# imports

import pandas as P
import scipy.stats as S
import numpy as N

<h2>Data preparation</h2>

In [ ]:
# loading data file

bikes = P.read_csv('./project_data/bikes.data',
                   )